In [2]:
!pip install transformers 

     |████████████████████████████████| 778kB 3.4MB/s 
     |████████████████████████████████| 3.0MB 24.6MB/s 
     |████████████████████████████████| 1.1MB 19.8MB/s 
     |████████████████████████████████| 890kB 38.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=7038b92e0347a92dc1010cbef618fd296ddd6932099997dcbfa37ad090cea5e0
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
 %reload_ext autoreload
 %autoreload 2
 %matplotlib inline

In [4]:
  

import pandas as pd
import math
import numpy as np
from sklearn.metrics import classification_report
import torch.nn.functional as F
import pandas as pd
import math
import numpy as np
from sklearn.metrics import classification_report
import torch.nn.functional as F
import torch
import os
from tqdm import tqdm,trange
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from transformers import (XLNetConfig, XLNetForSequenceClassification, XLNetTokenizer)



In [5]:
data_file = "/content/Training_Data_Set_for_Data_Science_Case_Study.xlsx"
df_data = pd.read_excel(data_file,encoding="latin1")
df_data.head(5)



,Company Name,Business Description,Industry Classification Tag
0,"ADSOUTH PARTNERS, INC.","Adsouth Partners, Inc. provides advertising ag...",Advertising
1,"Artec Global Media, Inc.","Artec Global Media, Inc., formerly Artec Consu...",Advertising
2,Betawave Corp.,Betawave Corporation provides online marketing...,Advertising
3,BOSTON OMAHA Corp,Boston Omaha Corporation is engaged in the bus...,Advertising
4,Bright Mountain Media Inc,"Bright Mountain Media, Inc. is a digital media...",Advertising


In [11]:
df_data = df_data.dropna()  #remove all
df_data["Business Description"] = df_data['Business Description'].str.replace('[^\w\s]','')

In [13]:
df_data["Business Description"] = df_data['Business Description'].str.lower()

In [23]:
spec_chars = ["!",'"',"#","%","&","'","(",")",
              "*","+",",","-",".","/",":",";","<",
              "=",">","?","@","[","\\","]","^","_",
              "`","{","|","}","~","–"]
for char in spec_chars:
    df_data['Business Description'] = df_data['Business Description'].str.replace(char, ' ')

In [24]:
df_data['Business Description'] = df_data['Business Description'].str.split().str.join(" ")

In [25]:
df_data["Business Description"]

0       adsouth partners inc provides advertising agen...
1       artec global media inc formerly artec consulti...
2       betawave corporation provides online marketing...
3       boston omaha corporation is engaged in the bus...
4       bright mountain media inc is a digital media h...
                              ...                        
6040    willis lease finance corporation along with it...
6041    windgen energy inc windgen is focused on wind ...
6042    xli technologies inc formerly mineria y explor...
6043    armco metals holdings inc is a scrap steel pro...
6044    foundation building materials inc is a special...
Name: Business Description, Length: 6022, dtype: object

In [26]:
df_data = df_data.dropna()
print(df_data['Business Description'].isnull().values.any())

print(df_data.columns)

label_list = df_data["Industry Classification Tag"].unique()
print(len(label_list))

print(df_data["Industry Classification Tag"].value_counts())

sentences = df_data["Business Description"].to_list()
print(sentences[0])

labels = df_data["Industry Classification Tag"].to_list()
print(len(labels))
print(labels[0])

False
Index(['Company Name', 'Business Description', 'Industry Classification Tag'], dtype='object')
62
Regional Banks                        569
Biotechnology                         402
Gold                                  357
Oil & Gas Exploration & Production    224
Application Software                  205
                                     ... 
Technology Distributors                51
Auto Parts & Equipment                 51
Internet Services & Infrastructure     50
Real Estate Operating Companies        50
Property & Casualty Insurance          50
Name: Industry Classification Tag, Length: 62, dtype: int64
adsouth partners inc provides advertising agency services specializing in direct response media campaigns it also owns the distribution rights to a number of products that it acquired the business consists of two operating sectors advertising and products during the year ended december 31 2005 it internally developed and marketed two lines of flashlights branded as the ex

In [27]:
tag2idx ={}
m= 0
for i in label_list:
    tag2idx[i] = m
    m = m+1

#print(tag2idx)

tag2name={tag2idx[key] : key for key in tag2idx.keys()}
print(tag2name)



{0: 'Advertising', 1: 'Aerospace & Defense', 2: 'Apparel Retail', 3: 'Apparel, Accessories & Luxury Goods', 4: 'Application Software', 5: 'Asset Management & Custody Banks', 6: 'Auto Parts & Equipment', 7: 'Biotechnology', 8: 'Building Products', 9: 'Casinos & Gaming', 10: 'Commodity Chemicals', 11: 'Communications Equipment', 12: 'Construction & Engineering', 13: 'Construction Machinery & Heavy Trucks', 14: 'Consumer Finance', 15: 'Data Processing & Outsourced Services', 16: 'Diversified Metals & Mining', 17: 'Diversified Support Services', 18: 'Electric Utilities', 19: 'Electrical Components & Equipment', 20: 'Electronic Equipment & Instruments', 21: 'Environmental & Facilities Services', 22: 'Gold', 23: 'Health Care Equipment', 24: 'Health Care Facilities', 25: 'Health Care Services', 26: 'Health Care Supplies', 27: 'Health Care Technology', 28: 'Homebuilding', 29: 'Hotels, Resorts & Cruise Lines', 30: 'Human Resource & Employment Services', 31: 'Industrial Machinery', 32: 'Integrat

In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
n_gpu


1

In [29]:
vocabulary = '/content/xlnet-base-cased-spiece.model'      # initially i am using pretrainde vocab for tokenization

max_len  = 128
tokenizer = XLNetTokenizer(vocab_file=vocabulary,do_lower_case=False)

max_len  =  128

full_input_ids = []
full_input_masks = []
full_segment_ids = []

SEG_ID_A   = 0
SEG_ID_B   = 1
SEG_ID_CLS = 2
SEG_ID_SEP = 3
SEG_ID_PAD = 4

UNK_ID = tokenizer.encode("<unk>")[0]  
CLS_ID = tokenizer.encode("<cls>")[0]
SEP_ID = tokenizer.encode("<sep>")[0]
MASK_ID = tokenizer.encode("<mask>")[0]
EOD_ID = tokenizer.encode("<eod>")[0]

In [30]:
for i,sentence in enumerate(sentences):
    # Tokenize sentence to token id list
    tokens_a = tokenizer.encode(sentence)
    
    # Trim the len of text
    if(len(tokens_a)>max_len-2):
        tokens_a = tokens_a[:max_len-2]
    
    tokens = []
    segment_ids = []
    
    for token in tokens_a:
        tokens.append(token)
        segment_ids.append(SEG_ID_A)
        
    tokens.append(SEP_ID)
    segment_ids.append(SEG_ID_A)
   
    tokens.append(CLS_ID)
    segment_ids.append(SEG_ID_CLS)
    
    input_ids = tokens
    
    input_mask = [0] * len(input_ids)  # mask 0 for real tokens and 1 for padding tokens
    
    if len(input_ids) < max_len:
        delta_len = max_len - len(input_ids)
        input_ids = [0] * delta_len + input_ids
        input_mask = [1] * delta_len + input_mask
        segment_ids = [SEG_ID_PAD] * delta_len + segment_ids

    
    full_input_ids.append(input_ids)
    full_input_masks.append(input_mask)
    full_segment_ids.append(segment_ids)
    

In [31]:
tags = [tag2idx[str(lab)] for lab in labels]
print(tags[300])

tr_inputs, val_inputs, tr_tags, val_tags,tr_masks, val_masks,tr_segs, val_segs = train_test_split(full_input_ids, tags,full_input_masks,full_segment_ids,random_state=4, test_size=0.3)


tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)
tr_segs = torch.tensor(tr_segs)
val_segs = torch.tensor(val_segs)






3


In [32]:
batch_num = 32

train_data = TensorDataset(tr_inputs, tr_masks,tr_segs, tr_tags)
train_sampler = RandomSampler(train_data)

train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_num,drop_last=True) # Drop last can make batch training better for the last one

valid_data = TensorDataset(val_inputs, val_masks,val_segs, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=batch_num)



In [33]:
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased",num_labels=len(tag2idx))

model;

model.to(device);

if n_gpu >1:
    model = torch.nn.DataParallel(model)

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [45]:
epochs = 5
max_grad_norm = 1.0
num_train_optimization_steps = int( math.ceil(len(tr_inputs) / batch_num) / 1) * epochs
True
FULL_FINETUNING = True   # you can fine tune all the parameters or only classfier layer with FULL_FINETUNING.

if FULL_FINETUNING:
    
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
   
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)





In [46]:
model.train()  #initiating training

print("  Num examples = %d"%(len(tr_inputs)))
print("  Batch size = %d"%(batch_num))
print("  Num steps = %d"%(num_train_optimization_steps))
for _ in trange(epochs,desc="Epoch"):
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
       
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_segs,b_labels = batch
        
        # ---------------forward prop-----------------
        outputs = model(input_ids =b_input_ids,token_type_ids=b_segs, input_mask = b_input_mask,labels=b_labels)
        loss, logits = outputs[:2]
        if n_gpu>1:
            loss = loss.mean()  # for multi-gup average it
        
        # ------------------backward prop------------
        loss.backward()
        
        #----training loss----
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        
        #-------- update parameters---------
        optimizer.step()
        optimizer.zero_grad()
        
        
    print("Train loss: {}".format(tr_loss/nb_tr_steps))


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

  Num examples = 4215
  Batch size = 32
  Num steps = 660


Epoch:  20%|██        | 1/5 [04:17<17:10, 257.53s/it]

Train loss: 0.1062272229722438


Epoch:  40%|████      | 2/5 [08:34<12:52, 257.36s/it]

...

Train loss: 0.040497852020136275


Epoch:  80%|████████  | 4/5 [17:08<04:17, 257.08s/it]

Train loss: 0.038283529456790165


Epoch: 100%|██████████| 5/5 [21:24<00:00, 257.00s/it]

Train loss: 0.031716345205106806


In [47]:
xlnet_out_address = '/content/sample_data/models/xlnet_out_model/tc02'

if not os.path.exists(xlnet_out_address):
        os.makedirs(xlnet_out_address)

model_to_save = model.module if hasattr(model, 'module') else model

output_model_file = os.path.join(xlnet_out_address, "pytorch_model.bin")
output_config_file = os.path.join(xlnet_out_address, "config.json")

torch.save(model_to_save.state_dict(), output_model_file)
model_to_save.config.to_json_file(output_config_file)
tokenizer.save_vocabulary(xlnet_out_address)


('/content/sample_data/models/xlnet_out_model/tc02/spiece.model',)

In [48]:
model = XLNetForSequenceClassification.from_pretrained(xlnet_out_address,num_labels=len(tag2idx))

model.to(device);

if n_gpu >1:
    model = torch.nn.DataParallel(model)

model.eval();

def accuracy(out, labels):
    outputs = np.argmax(out, axis=1)
    return np.sum(outputs == labels)

eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

y_true = []
y_predict = []

In [49]:
print("  Num examples ={}".format(len(val_inputs)))
print("  Batch size = {}".format(batch_num))
for step, batch in enumerate(valid_dataloader):
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_segs,b_labels = batch
    
    with torch.no_grad():
        outputs = model(input_ids =b_input_ids,token_type_ids=b_segs, input_mask = b_input_mask,labels=b_labels)
        tmp_eval_loss, logits = outputs[:2]# outputs in a tuple form 
    
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    tmp_eval_accuracy = accuracy(logits, label_ids)

    for predict in np.argmax(logits, axis=1):
        y_predict.append(predict)
        
    for real_result in label_ids.tolist():
        y_true.append(real_result)

    
    eval_loss += tmp_eval_loss.mean().item()
    eval_accuracy += tmp_eval_accuracy
   
    nb_eval_steps += 1

eval_loss = eval_loss / nb_eval_steps
eval_accuracy = eval_accuracy / len(val_inputs)
loss = tr_loss/nb_tr_steps 
result = {'eval_loss': eval_loss,
                  'eval_accuracy': eval_accuracy,
                  'loss': loss}
report = classification_report(y_pred=np.array(y_predict),y_true=np.array(y_true))


  Num examples =1807
  Batch size = 32


In [43]:
print(report)

              precision    recall  f1-score   support

           0       0.59      0.59      0.59        34
           1       0.71      0.63      0.67        19
           2       0.65      0.71      0.68        21
           3       0.63      0.81      0.71        21
           4       0.59      0.63      0.61        65
           5       0.74      0.46      0.57        37
           6       0.69      0.55      0.61        20
           7       0.76      0.91      0.83       120
           8       0.81      0.58      0.68        36
           9       0.97      0.97      0.97        29
          10       0.67      0.36      0.47        22
          11       0.74      0.63      0.68        27
          12       0.67      0.43      0.53        23
          13       0.41      0.54      0.47        13
          14       0.53      0.64      0.58        14
          15       0.75      0.45      0.56        20
          16       0.50      0.33      0.40        12
          17       0.50    

In [50]:
output_eval_file = os.path.join(xlnet_out_address, "eval_results.txt")
with open(output_eval_file, "w") as writer:
    for key in sorted(result.keys()):
        print("  %s = %s"%(key, str(result[key])))
        writer.write("%s = %s\n" % (key, str(result[key])))
        
    print(report)
    writer.write("\n\n")  
    writer.write(report)


  eval_accuracy = 0.7172108467072495
  eval_loss = 1.711126724878947
  loss = 0.031716345205106806
              precision    recall  f1-score   support

           0       0.65      0.71      0.68        34
           1       0.82      0.74      0.78        19
           2       0.78      0.67      0.72        21
           3       0.55      0.86      0.67        21
           4       0.61      0.58      0.60        65
           5       0.73      0.65      0.69        37
           6       0.57      0.40      0.47        20
           7       0.89      0.69      0.78       120
           8       0.77      0.64      0.70        36
           9       0.97      0.97      0.97        29
          10       0.41      0.41      0.41        22
          11       0.79      0.70      0.75        27
          12       0.80      0.52      0.63        23
          13       0.60      0.46      0.52        13
          14       0.55      0.86      0.67        14
          15       0.92      0.55   